# Spark History Server (SHS)

## Мотивация

Анализ производительности обычно выполняют при помощи Spark Web UI (Spark UI), который традиционно стартует на порту 4040 машины, на которой запущено приложение Apache Spark. Основная проблема при этом заключается в том, что Spark UI доступен пока доступно приложение. После завершения работы приложения все данные пропадают, что делает невозможным ретроспективный анализ приложений.

**Spark History Server (SHS)** призван решить эту проблему. Основная идея заключается в том, что приложение Spark генерирует события, а внешний по отношению к приложению Spark сервер (**Spark History Server**) считывает эти события и визуализирует их аналогично Spark UI. Основное требование - выгрузка событий в директорию, которая доступна **Spark History Server**. Для этих целей отлично подходит HDFS или S3. По умолчанию **SHS** стартует на порту `18080`.

Внешняя директория для логов должна быть создана заранее:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -mkdir -p /shared/spark-logs

## Конфигурация

Чтобы начать использовать **Spark History Server**, необходимо добавить дополнительную конфигурацию как на стороне приложения, так и на стороне **Spark History Server**.

### Конфигурация Spark приложения

Необходимо разрешить **Spark** приложению логировать события, а также указать место, куда эти логи складывать:

- `spark.eventLog.enabled` разрешает логирование событий,
- `spark.eventLog.dir` указывает директорию для логов.

### Конфигурация Spark History Server

**Spark History Server** должен знать, где находятся файлы с логами приложения. Для этого необходимо установить значение параметра `spark.history.fs.logDirectory`.

### Конфигурация приложений по умолчанию

Традиционно параметры приложения указываются при помощи [`SparkSession.Builder#config`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.builder.config.html) при запуске приложения. Но это не единственный вариант передачи параметров, параметры можно передать через файл `$SPARK_HOME/conf/spark-defaults.conf`:

In [ ]:
! cat ${SPARK_HOME}/conf/spark-defaults.conf

Файл `$SPARK_HOME/conf/spark-defaults.conf` представляет собой текстовый файл в формате [`*.properties`](https://ru.wikipedia.org/wiki/.properties). При этом в дополнении к стандартному формату, можно использовать переменные окружения или системные переменные в значениях. Формат параметров можно найти в [документации](https://github.com/apache/spark/blob/39cc4abaff73cb49f9d79d1d844fe5c9fa14c917/sql/core/src/main/scala/org/apache/spark/sql/internal/VariableSubstitution.scala#L25).

Параметры из файла `$SPARK_HOME/conf/spark-defaults.conf` будут автоматически добавлены к конфигурации каждого приложения, запускаемого на текущей машине. Файл `$SPARK_HOME/conf/spark-defaults.conf` может отсутствовать, пользователь при желании может создать этот файл самостоятельно.

## Запуск приложения

При корректной конфигурации **Spark History Server** будет не важно в каком режиме запущено приложение Spark: Local, Yarn, Mesos, k8s. Единственное требование, как было указано выше, $ - $ это выгрузка логов во внешнюю директорию (HDFS, S3 и т.д.).

In [ ]:
from pyspark.sql import SparkSession

### Запуск приложения в локальном режиме

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Local History Server")
        .master("local")
        .getOrCreate()
)
sc = spark.sparkContext

In [ ]:
external_dir = spark.conf.get("spark.eventLog.dir")
status = f"{external_dir}" if spark.conf.get("spark.eventLog.enabled") == 'true' else "запрещен"

print(f"Экспорт логов: {status}")

Команда выше показывает, что выгрузка логов выполняется в директорию `/shared/spark-logs` на HDFS, проверим состояние внешней директории:

In [ ]:
! source ~/.bash_aliases && \
hdfs dfs -ls /shared/spark-logs

Если открыть [`18080`](http://localhost:18080), то можно увидеть как завершенные, так и незавершенные приложения:

- [завершенные приложения](http://localhost:18080/?showIncomplete=false),
- [незавершенные приложения](http://localhost:18080/?showIncomplete=true).

Среди [незавершенных приложений](http://localhost:18080/?showIncomplete=true) можно увидеть приложение с именем (колонка `App Name`) **Local History Server** $ - $ текущее запущенное приложение.

Запустим нагрузку:

In [ ]:
df = spark.sql("select 'Hello, Local Mode Spark History Server!' as message")

In [ ]:
df.show(1, False)

In [ ]:
spark.stop()

### Запуск приложения на Yarn

В качестве демонстрации независимости **SHS** от режима работы приложений Spark, запустим приложение на `Yarn`:

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("Yarn Spark History Server")
        .master("yarn")
        .getOrCreate()
)
sc = spark.sparkContext

Среди [незавершенных приложений](http://localhost:18080/?showIncomplete=true) можно увидеть приложение с именем (колонка `App Name`) **Yarn Spark History Server** $ - $ текущее запущенное приложение.

И выполним нагрузку:

In [ ]:
df = spark.sql("select 'Hello, Yarn Mode Spark History Server!' as message")

In [ ]:
df.show(5, False)

In [ ]:
spark.stop()

## Запуск Spark History Server

Может сложиться впечателние, что **Spark History Server** стартует автоматически при старте Spark приложения, но это не так. **Spark History Server** запускается при помощи команды `$SPARK_HOME/sbin/start-history-server.sh`:

In [ ]:
! ls -l $SPARK_HOME/sbin/start-history-server.sh

Текущий сетап организован таким образом, что среди docker сервисов есть сервис, который автоматически стартует **Spark History Server**:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps historyserver1

Процессы, запущенные в контейнере `historyserver1`:

In [ ]:
! source ~/.bash_aliases && \
docker compose top historyserver1

Файл `/bin/entrypoint` является точкой входа (entrypoint) в контейнер, и запускается при старте контейнера.

При старте контейнера `historyserver1` выполняется инициализация контейнера, а также и запуск **Spark History Server** при помощи команды `start-history-server.sh`:

In [ ]:
! source ~/.bash_aliases && HOST=historyserver1 execute \
grep 'start-history-server.sh' /bin/entrypoint

## Высокая доступность Spark History Server $ - $ Spark History Server High Availability

**Spark History Server** не хранит состояние, вся его работа заключается в считывании событий с внешней директории и отрисовки Web UI. Поэтому для достижения высокой доступности (High Availability) можно просто запустить несколько **Spark History Server** серверов и поставить перед ними **Load Balancer**, например, `nginx`.

**Nginx** в этом случае может по кругу (Round Robin) перебирать запущенные **Spark History Server** серверы и направлять поступающие запросы. Текущий сетап содержит три **Spark History Server** сервиса и один сервис **Nginx** для балансировки нагрузки:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps nginx historyserver1 historyserver2 historyserver3

**Nginx** запущен на порту [8080](http://localhost:8080), поэтому можно работать с **SHS** через сервис `nginx`.

### Симуляция сбоев Spark History Server

**Spark History Server** может выйти из строя в любой момент, поэтому очень важно продолжать обслуживать клиентов независимо от сбоев. Так, если часть серверов **Spark History Server** доступны, то необходимо перенаправлять запросы им.

В качестве симуляции сбоя поставим `historyserver1` на паузу:

In [ ]:
! source ~/.bash_aliases && \
docker compose pause historyserver1

Сервис `historyserver1` находится на паузе:

In [ ]:
! source ~/.bash_aliases && \
docker compose ps nginx historyserver1 historyserver2 historyserver3

При этом запросы к `nginx` по порту [8080](http://localhost:8080) по прежнему отрабатывают. В демонстрационных целях таймаут на ответ от сервера ограничен пятью секундами, поэтому необхдимо подождать, т.к. `nginx` автоматически будет перенаправлять запросы живым серверам при наступлении таймаута. По умолчанию Nginx использует 120 секунд (две минуты) в качестве значения таймаута.

**Spark History Server** запущен на трех сервисах, а значит вполне можно потерять 2 сервиса без вреда для клиентов. В качестве симуляции нового сбоя поставим `historyserver2` на паузу:

In [ ]:
! source ~/.bash_aliases && \
docker compose pause historyserver2

Теперь два из трёх **SHS** сервисов находятся на паузе (симулируют сбой):

In [ ]:
! source ~/.bash_aliases && \
docker compose ps nginx historyserver1 historyserver2 historyserver3

При этом запросы к `nginx` по порту [8080](http://localhost:8080) по прежнему отрабатывают: все запросы уходят на сервис `historyserver3`.

Если все три сервиса выйдут из строя, то **Spark History Server** перестанет быть доступным для клиентов. В качестве симуляции нового сбоя поставим `historyserver3` на паузу:

In [ ]:
! source ~/.bash_aliases && \
docker compose pause historyserver3

Сейчас все три **SHS** сервиса находятся на паузе (симулируют сбой):

In [ ]:
! source ~/.bash_aliases && \
docker compose ps nginx historyserver1 historyserver2 historyserver3

Запросы к `nginx` по порту [8080](http://localhost:8080) невозможно обработать (`504 Gateway Time-out`), т.к. нет ни одного доступного **SHS** сервиса.

Восставновление работы **SHS** сервисов вернет клиентам доступ к **Spark History Server**:

In [ ]:
! source ~/.bash_aliases && \
docker compose unpause historyserver1 historyserver2 historyserver3

Все три **SHS** сервиса доступны (сняты с паузы):

In [ ]:
! source ~/.bash_aliases && \
docker compose ps nginx historyserver1 historyserver2 historyserver3

Запросы к `nginx` по порту [8080](http://localhost:8080) обрабатываются в штатном режиме.

## Выводы

Анализ рабочей нагрузки (workload) играет ключевую роль в оптимизации производительности запросов. Очень важно уметь анализировать как активную нагрузку, так и исторические показатели. **Spark History Server** позволяет решить задачу анализа активных и ретроспективных задач. Основной недостаток **Spark History Server** заключается в небольшом оставании по времени визуализируемых данных, т.к. **SHS** необходимо обнаружить новые данные, а также потратить некоторое время на разбор файлов с логами. С другой стороны надежность **SHS** и простота его запуска в режиме высокой доступности делает его пригодным для эксплуатации в продуктовом окружении.

## Задание

1. Запустить приложение со своим именем в режиме Yarn.

<details>
    <summary>Ответ</summary>

1. Запустить
```python
spark = (
    SparkSession
        .builder
        .appName("Nikita's Yarn Spark Application")
        .master("yarn")
        .getOrCreate()
)
```

</details>

2. Как через **Spark History Server** определить сколько воркеров доступно приложению?

<details>
    <summary>Ответ</summary>

1. Найти приложение `Nikita's Yarn Spark Application` в списке приложений http://localhost:8080/
1. Открыть вкладку Executors
1. В разделе Executors в таблице находится три строки: 1 драйвер и 2 воркера

**Ответ**: 2 воркера доступны приложению.
</details>